In [63]:
import os
from glob import glob

import numpy as np
from scipy.spatial.distance import cdist

import trimesh
#requires shapely, rtree, networkx, pyglet
import pyvista as pv

import mbb

In [2]:
"""
This notebook is for a new method to simulate the manual measurements of hailstones on 3D models.
This includes the measurement of Dmax, Dint and Dmin following the standard procedure.
The implementation is heavily optimised to use the trimesh library and convexhulls.

Summary of method:

(1) Shift model centre of mass to global coordinates 0,0,0
(2) calculate 3D convex hull of model
(3) calculate which pair of vertices in 3D convex hull have the greatest separation (Dmax)
(4) Calculate normal plane to Dmax (Dint-Dmin plane), and the mid point
(5) Slice the convex hull at this mid point using the Dint-Dmin plane
(6) Fit a 2D minimum bounded box to the convex hull slice to find Dint and Dmin.

"""

'\nThis notebook is for a new method to simulate the manual measurements of hailstones on 3D models.\nThis includes the measurement of Dmax, Dint and Dmin following the standard procedure.\nThe implementation is heavily optimised to use the trimesh library and convexhulls.\n\nSummary of method:\n\n(1) Shift model centre of mass to global coordinates 0,0,0\n(2) calculate 3D convex hull of model\n(3) calculate which pair of vertices in 3D convex hull have the greatest separation (Dmax)\n(4) Calculate normal plane to Dmax (Dint-Dmin plane), and the mid point\n(5) Slice the convex hull at this mid point using the Dint-Dmin plane\n(6) Fit a 2D minimum bounded box to the convex hull slice to find Dint and Dmin.\n\n'

In [106]:
def slice_midpoint_and_measure(tmp_mesh, midpoint, nvec):

    # slice mesh at midpoint along normal vector,
    # fit with bounding box and measure minimum (Dmin) and maximum (Dint) 

    #create slice
    slice = tmp_mesh.section(plane_origin=midpoint, 
                        plane_normal=nvec)
    #convert to 2D projection
    slice_2D, to_3D = slice.to_2D(normal=nvec)
    #generate bounding box on 2D projection
    bounding_box_stats = mbb.MinimumBoundingBox(slice_2D.vertices)
    #use bounding box to calculate Dint and Dmin
    Dint = bounding_box_stats.length_parallel
    Dmin = bounding_box_stats.length_orthogonal
    #extract corner points of bounding box and order
    corner_points = np.array(list(bounding_box_stats.corner_points))
    pts_order = trimesh.points.tsp(corner_points, start=0)[0]
    corner_points = corner_points[pts_order]
    #create a new 2Dpath object for corner points, using transform from slice
    slice_bb = trimesh.path.Path2D(entities=[trimesh.path.entities.Line([0,1,2,3,0])], vertices=corner_points).to_3D(transform=to_3D)

    return slice, slice_bb, Dint, Dmin


def measure_shape(stl_ffn):

    #main function to measure Dmax, Dint, Dmin from a STL file

    #load mesh
    hail_mesh = trimesh.load_mesh(stl_ffn)
    # volumetric center of mass which we can set as the origin for our mesh
    hail_mesh.vertices -= hail_mesh.center_mass
    if not hail_mesh.is_watertight:
        print('WARNING, MESH is not watertight!, stats may be misleading')

    #get centre of mass and volume
    com = hail_mesh.center_mass
    volume = hail_mesh.volume

    #derive convex hull model
    hail_mesh_hull = hail_mesh.convex_hull

    # Naive way of finding the best pair in O(H^2) time if H is number of points on hull
    Dmax_hdist = cdist(hail_mesh_hull.vertices, hail_mesh_hull.vertices, metric='euclidean')
    Dmax_hull = Dmax_hdist.max()
    Dmax_bestpair = np.unravel_index(Dmax_hdist.argmax(), Dmax_hdist.shape)
    Dmax_points = [hail_mesh_hull.vertices[Dmax_bestpair[0]], hail_mesh_hull.vertices[Dmax_bestpair[1]]]

    #calculate mid point and normal vector
    Dmax_midpoint = (Dmax_points[0] + Dmax_points[1])/2
    #calculate normal vector between Dmax points
    Dint_Dmin_plane_nvec = Dmax_points[0] - Dmax_points[1]
    

    #1: slice along Dint/Dmin plane
    #myslice, slice_bb, Dint, Dmin = slice_midpoint_and_measure(hail_mesh, Dmax_midpoint, Dint_Dmin_plane_nvec)
    #2: slice convex hull along Dint/Dmin plane
    hail_mesh_hull_slice, hail_mesh_hull_bb, Dint_hull, Dmin_hull = slice_midpoint_and_measure(hail_mesh_hull, Dmax_midpoint, Dint_Dmin_plane_nvec)
    #3: slice along Dmax/Dmin plane
    Dmax_Dmin_plane_nvec = hail_mesh_hull_bb.vertices[0,:] - hail_mesh_hull_bb.vertices[1,:]
    hail_mesh_hull_slice_dmax, _, _, _ = slice_midpoint_and_measure(hail_mesh_hull, Dmax_midpoint, Dmax_Dmin_plane_nvec)

    return Dmax_hull, Dint_hull, Dmin_hull, hail_mesh, hail_mesh_hull, hail_mesh_hull_slice, hail_mesh_hull_bb, hail_mesh_hull_slice_dmax

In [111]:
stl_ffn = 'gatton_20231223_2-1.stl'
Dmax_hull, Dint_hull, Dmin_hull, hail_mesh, hail_mesh_hull, hail_mesh_hull_slice, hail_mesh_hull_bb, hail_mesh_hull_slice_dmax = measure_shape(stl_ffn)
print(stl_ffn, 'convex hull dmax:', Dmax_hull, 'dint:', Dint_hull, 'dmin:', Dmin_hull)

gatton_20231223_2-1.stl convex hull dmax: 92.83988307222258 dint: 78.53321502531429 dmin: 45.82452505057957


In [ ]:
import trimesh
import k3d
import numpy as np

plot = k3d.plot()

# Plot hailstone
plot += k3d.mesh(
    hail_mesh.vertices.astype(np.float32),
    hail_mesh.faces.astype(np.uint32),
    color=0xffffff,  # White
    opacity=1.0
)

#plot convex hull
plot += k3d.mesh(
    hail_mesh_hull.vertices.astype(np.float32), 
    hail_mesh_hull.faces.astype(np.uint32),
    color=0x00ff00,  # Green
    opacity=0.2
)

#plot Dmax plane (black)
for entity in hail_mesh_hull_slice_dmax.entities:
    if hasattr(entity, 'points'):
        # Get points in the order defined by the entity
        points = hail_mesh_hull_slice_dmax.vertices[entity.points]
        
        # Close the loop by adding first point at end
        if len(points) > 2:
            closed_points = np.vstack([points, points[0]])
            
            plot += k3d.line(
                closed_points.astype(np.float32),
                color=0x000000,
                width=1
            )

#plot Dint-Dmin plane (green)
for entity in myslice_hull.entities:
    if hasattr(entity, 'points'):
        # Get points in the order defined by the entity
        points = myslice_hull.vertices[entity.points]
        
        # Close the loop by adding first point at end
        if len(points) > 2:
            closed_points = np.vstack([points, points[0]])
            
            plot += k3d.line(
                closed_points.astype(float),
                color=0x00ff00,
                width=1
            )

#plot Dint-Dmin bounding box (blue)
for entity in slice_bb_hull.entities:
    if hasattr(entity, 'points'):
        # Get points in the order defined by the entity
        points = slice_bb_hull.vertices[entity.points]
        
        # Close the loop by adding first point at end
        if len(points) > 2:
            closed_points = np.vstack([points, points[0]])
            
            plot += k3d.line(
                closed_points.astype(float),
                color=0x0000ff,
                width=1
            )

plot.display()

Output()